In [1]:
from types import SimpleNamespace
from collections import Counter
import os
import re
import pathlib
import array
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import math

In [2]:
DATASET_VERSION = 'ca-100'
COMPETITION_ROOT = '../input/vectors4'
DATASET_ROOT = f'../input/text-preprocessing/data/{DATASET_VERSION}'
WORKING_ROOT = f'data/{DATASET_VERSION}'
DATASET_PREFIX = 'ca.wiki'

In [3]:
params = SimpleNamespace(
    embedding_dim = 300,
    window_size = 7,
    batch_size = 2048,
    epochs = 4,
    preprocessed = f'{DATASET_ROOT}/{DATASET_PREFIX}',
    working = f'{WORKING_ROOT}/{DATASET_PREFIX}',
    modelname = f'{WORKING_ROOT}/{DATASET_VERSION}.pt',
    train = True,
)

In [4]:
class Vocabulary(object):
    def __init__(self, pad_token='<pad>', unk_token='<unk>', eos_token='<eos>'):
        self.token2idx = {}
        self.idx2token = []
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_token = eos_token
        if pad_token is not None:
            self.pad_index = self.add_token(pad_token)
        if unk_token is not None:
            self.unk_index = self.add_token(unk_token)
        if eos_token is not None:
            self.eos_index = self.add_token(eos_token)

    def add_token(self, token):
        if token not in self.token2idx:
            self.idx2token.append(token)
            self.token2idx[token] = len(self.idx2token) - 1
        return self.token2idx[token]

    def get_index(self, token):
        if isinstance(token, str):
            return self.token2idx.get(token, self.unk_index)
        else:
            return [self.token2idx.get(t, self.unk_index) for t in token]

    def get_token(self, index):
        return self.idx2token[index]

    def __len__(self):
        return len(self.idx2token)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.__dict__.update(pickle.load(f))

In [5]:
def batch_generator(idata, target, batch_size, shuffle=True):
    nsamples = len(idata)
    if shuffle:
        perm = np.random.permutation(nsamples)
    else:
        perm = range(nsamples)

    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            yield idata[batch_idx], target[batch_idx]
        else:
            yield idata[batch_idx], None

In [6]:
def load_preprocessed_dataset(prefix):
    # Try loading precomputed vocabulary and preprocessed data files
    token_vocab = Vocabulary()
    token_vocab.load(f'{prefix}.vocab')
    data = []
    for part in ['train', 'valid', 'test']:
        with np.load(f'{prefix}.{part}.npz') as set_data:
            idata, target = set_data['idata'], set_data['target']
            data.append((idata, target))
            print(f'Number of samples ({part}): {len(target)}')
    print("Using precomputed vocabulary and data files")
    print(f'Vocabulary size: {len(token_vocab)}')
    return token_vocab, data

In [7]:
def train(model, criterion, optimizer, idata, target, batch_size, device, log=False):
    model.train()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    for X, y in batch_generator(idata, target, batch_size, shuffle=True):
        # Get input and target sequences from batch
        X = torch.tensor(X, dtype=torch.long, device=device)
        y = torch.tensor(y, dtype=torch.long, device=device)

        model.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        # Training statistics
        total_loss += loss.item()
        ncorrect += (torch.max(output, 1)[1] == y).sum().item()
        ntokens += y.numel()
        niterations += 1
        if niterations == 200 or niterations == 500 or niterations % 1000 == 0:
            print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={100*ncorrect/ntokens:.1f}, loss={total_loss/ntokens:.2f}')

    total_loss = total_loss / ntokens
    accuracy = 100 * ncorrect / ntokens
    if log:
        print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={accuracy:.1f}, loss={total_loss:.2f}')
    return accuracy, total_loss

In [8]:
def validate(model, criterion, idata, target, batch_size, device):
    model.eval()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    y_pred = []
    with torch.no_grad():
        for X, y in batch_generator(idata, target, batch_size, shuffle=False):
            # Get input and target sequences from batch
            X = torch.tensor(X, dtype=torch.long, device=device)
            output = model(X)
            if target is not None:
                y = torch.tensor(y, dtype=torch.long, device=device)
                loss = criterion(output, y)
                total_loss += loss.item()
                ncorrect += (torch.max(output, 1)[1] == y).sum().item()
                ntokens += y.numel()
                niterations += 1
            else:
                pred = torch.max(output, 1)[1].detach().to('cpu').numpy()
                y_pred.append(pred)

    if target is not None:
        total_loss = total_loss / ntokens
        accuracy = 100 * ncorrect / ntokens
        return accuracy, total_loss
    else:
        return np.concatenate(y_pred)

In [9]:
# Create working dir
pathlib.Path(WORKING_ROOT).mkdir(parents=True, exist_ok=True)

In [10]:
# Select device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("WARNING: Training without GPU can be very slow!")

In [11]:
vocab, data = load_preprocessed_dataset(params.preprocessed)

Number of samples (train): 82284341
Number of samples (valid): 164765
Number of samples (test): 165837
Using precomputed vocabulary and data files
Vocabulary size: 100002


In [12]:
# 'El Periodico' validation dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_valid.csv')
tokens = valid_x_df.columns[1:]
valid_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')
valid_y_df = pd.read_csv(f'{COMPETITION_ROOT}/y_valid.csv')
valid_y = valid_y_df['token'].apply(vocab.get_index).to_numpy(dtype='int32')

In [13]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float('-Inf'))
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value)#, p_attn

The **xavier_normal_()** initialization function fills the input Tensor with values according to the method described in **Understanding the difficulty of training deep feedforward neural networks** - Glorot, X. & Bengio, Y. (2010), using a uniform distribution. The resulting tensor will have values sampled from a uniform distribution \\(\mathcal{U}(-a, a)\\) where

\\(    a = \text{gain} \times \sqrt{\frac{6}{\text{fan\_in} + \text{fan\_out}}}\\)

Also known as Glorot initialization.

In [14]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, bias=True):
        super().__init__()
        self.k_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.v_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.q_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        # Empirically observed the convergence to be much better with the scaled initialization
        nn.init.xavier_uniform_(self.k_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.v_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.q_proj.weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.out_proj.weight)
        if self.out_proj.bias is not None:
            nn.init.constant_(self.out_proj.bias, 0.)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    def forward(self, x):
        # x shape is (B, W, E)
        q = self.q_proj(x)
        # q shape is (B, W, E)
        k = self.k_proj(x)
        # k shape is (B, W, E)
        v = self.v_proj(x)
        # k shape is (B, W, E)
        y, _ = attention(q, k, v)
        # y shape is (B, W, E)
        y = self.out_proj(y)
        # y shape is (B, W, E)
        return y

In [15]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, embed_dim, num_heads=4, bias=True):
        super().__init__()
        
        assert(embed_dim%num_heads==0)
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        
        self.k_proj = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=bias) for i in range (num_heads)])
        self.v_proj = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=bias) for i in range (num_heads)])
        self.q_proj = nn.ModuleList([nn.Linear(self.head_dim, self.head_dim, bias=bias) for i in range (num_heads)])
        self.out_proj = nn.Linear(embed_dim, embed_dim, bias=bias)
        self.reset_parameters()
    
    def reset_parameters(self):
        # Empirically observed the convergence to be much better with the scaled initialization
        for i in range (self.num_heads):
            nn.init.xavier_uniform_(self.k_proj[i].weight, gain=1 / math.sqrt(2))
            nn.init.xavier_uniform_(self.q_proj[i].weight, gain=1 / math.sqrt(2))
            nn.init.xavier_uniform_(self.v_proj[i].weight, gain=1 / math.sqrt(2))
        nn.init.xavier_uniform_(self.out_proj.weight)
        if self.out_proj.bias is not None:
            nn.init.constant_(self.out_proj.bias, 0.)

    def forward(self, x):
        # x shape is (B, W, E)
        x = torch.split(x, self.head_dim, dim=2)
        q = [self.q_proj[i](x[i]) for i in range(self.num_heads)]
        k = [self.k_proj[i](x[i]) for i in range(self.num_heads)]
        v = [self.v_proj[i](x[i]) for i in range(self.num_heads)]

        # Determine output
        y = [attention(q[i], k[i], v[i]) for i in range(self.num_heads)]
        # Concatenar aquí
        y = torch.cat(y, dim=2)
        y = self.out_proj(y)
        return y

In [16]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, dim_feedforward=512, dropout=0.1, activation="relu"):
        super().__init__()
        # Multi Head Attention
        self.self_attn = MultiHeadAttention(d_model)
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

In [17]:
class Predictor(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, context_words=params.window_size-1):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        self.att1 = TransformerLayer(embedding_dim)
        self.att2 = TransformerLayer(embedding_dim)
        self.att3 = TransformerLayer(embedding_dim)
        self.meanpool = nn.AvgPool1d(kernel_size=context_words, stride=context_words)
        self.position_embedding = nn.Parameter(torch.Tensor(context_words, embedding_dim))
        nn.init.xavier_uniform_(self.position_embedding)

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        # e shape is (B, W, E)
        u = e + self.position_embedding
        # u shape is (B, W, E)
        
        v = self.att1(u)
        v2 = self.att2(v)
        v3 = self.att3(v2)
        
        x = self.meanpool(v3.transpose(1,2)).squeeze()
        # x shape is (B, E)
        
        y = self.lin(x)
        # y shape is (B, V)
        return y

In [18]:
model = Predictor(len(vocab), params.embedding_dim).to(device)

In [19]:
print(model)
for name, param in model.named_parameters():
    print(f'{name:20} {param.numel()} {list(param.shape)}')
print(f'TOTAL                {sum(p.numel() for p in model.parameters())}')

Predictor(
  (emb): Embedding(100002, 300, padding_idx=0)
  (lin): Linear(in_features=300, out_features=100002, bias=False)
  (att): TransformerLayer(
    (self_attn): MultiHeadAttention(
      (k_proj): ModuleList(
        (0): Linear(in_features=75, out_features=75, bias=True)
        (1): Linear(in_features=75, out_features=75, bias=True)
        (2): Linear(in_features=75, out_features=75, bias=True)
        (3): Linear(in_features=75, out_features=75, bias=True)
      )
      (v_proj): ModuleList(
        (0): Linear(in_features=75, out_features=75, bias=True)
        (1): Linear(in_features=75, out_features=75, bias=True)
        (2): Linear(in_features=75, out_features=75, bias=True)
        (3): Linear(in_features=75, out_features=75, bias=True)
      )
      (q_proj): ModuleList(
        (0): Linear(in_features=75, out_features=75, bias=True)
        (1): Linear(in_features=75, out_features=75, bias=True)
        (2): Linear(in_features=75, out_features=75, bias=True)
        

In [20]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(reduction='sum')

train_accuracy = []
wiki_accuracy = []
valid_accuracy = []
for epoch in range(params.epochs):
    acc, loss = train(model, criterion, optimizer, data[0][0], data[0][1], params.batch_size, device, log=True)
    train_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | train accuracy={acc:.1f}%, train loss={loss:.2f}')
    acc, loss = validate(model, criterion, data[1][0], data[1][1], params.batch_size, device)
    wiki_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (wikipedia)')
    acc, loss = validate(model, criterion, valid_x, valid_y, params.batch_size, device)
    valid_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (El Periódico)')

# Save model
torch.save(model.state_dict(), params.modelname)

Train: wpb=2048, num_updates=200, accuracy=7.4, loss=7.55
Train: wpb=2048, num_updates=500, accuracy=15.9, loss=6.54
Train: wpb=2048, num_updates=1000, accuracy=21.9, loss=5.85
Train: wpb=2048, num_updates=2000, accuracy=26.9, loss=5.29
Train: wpb=2048, num_updates=3000, accuracy=29.5, loss=5.00
Train: wpb=2048, num_updates=4000, accuracy=31.2, loss=4.82
Train: wpb=2048, num_updates=5000, accuracy=32.4, loss=4.68
Train: wpb=2048, num_updates=6000, accuracy=33.3, loss=4.58
Train: wpb=2048, num_updates=7000, accuracy=34.1, loss=4.49
Train: wpb=2048, num_updates=8000, accuracy=34.7, loss=4.42
Train: wpb=2048, num_updates=9000, accuracy=35.3, loss=4.36
Train: wpb=2048, num_updates=10000, accuracy=35.8, loss=4.30
Train: wpb=2048, num_updates=11000, accuracy=36.2, loss=4.25
Train: wpb=2048, num_updates=12000, accuracy=36.6, loss=4.21
Train: wpb=2048, num_updates=13000, accuracy=36.9, loss=4.17
Train: wpb=2048, num_updates=14000, accuracy=37.2, loss=4.14
Train: wpb=2048, num_updates=15000, ac

In [21]:
# 'El Periodico' test dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_test.csv')
test_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')
y_pred = validate(model, None, test_x, None, params.batch_size, device)
y_token = [vocab.get_token(index) for index in y_pred]

In [22]:
submission = pd.DataFrame({'id':valid_x_df['id'], 'token': y_token}, columns=['id', 'token'])
print(submission.head())
submission.to_csv('submission.csv', index=False)

   id  token
0   0      l
1   1      s
2   2  haver
3   3     no
4   4     el
